# The Battle of Neighborhoods

### Course: Applied Data Science Capstone

This notebook would contain Explaination and code for the coursera Capstone Project.

#### Adding Required Libraries to create required Environment

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

pd.set_option('display.expand_frame_repr', False)

# Segmenting and Clustering Neighborhoods in Toronto

### 1. Data scraping

#### Description:

Code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [4]:
# define the dataframe columns
column_names = ['Postal_Code','Borough', 'Neighborhood'] 

# instantiate the dataframe
nbr = pd.DataFrame(columns=column_names)

In [5]:
nbr

,Postal_Code,Borough,Neighborhood


##### Web scraping

##### Importing Data From provided URL

In [6]:
#import the library used to query a website
from urllib.request import urlopen

#specify the url
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#Query the website and return the wiki to the variable 'page'
page = urlopen(wiki)

#import the Beautiful soup functions to parse the data returned from the website
from bs4 import BeautifulSoup

#Parse the html in the 'page' variable, and store it in Beautiful Soup format
soup = BeautifulSoup(page, "lxml")

In [7]:
# Uncomment below to Print HTML of the Page
#print(soup.prettify()) 

#Get table 
right_table=soup.find('table', class_='wikitable sortable')
# uncomment below to print table
#right_table

##### Extract the information to DataFrame:

We iterate through each row (tr) and then assign each element of tr (td) to a variable and append it to a list.

In [8]:
#Generate lists
A=[]
B=[]
C=[]

for row in right_table.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

        
#Adding Data to our DataFrame
nbr['Postal_Code']=A
nbr['Borough']=B
nbr['Neighborhood']=C

#uncomment below to view dataframe nbr which contains data as retreived from the webpage in wiki
#nbr

##### Ignore cells with a borough that is Not assigned.

1. Compare with Each row if Borough is Not Assigned drop row.
2. Reset Index.

In [9]:
nbr = nbr.drop(nbr[nbr['Borough'].str.contains("Not assigned")==True].index, axis=0, inplace=False)

#Reset Index
nbr.index = pd.RangeIndex(len(nbr.index))

#Uncomment below to view Dataframe where not Assigned Boroughs have been dropped out.
#nbr

##### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

1. Instantiate additional dataframe nbrNA
2. Compare with Each row if Neighborhood is Not Assigned add value from Borough.

In [10]:
# instantiate additional dataframe nbrNA
column_n = ['NBRVAL'] 
nbrNA = pd.DataFrame(columns=column_n)
D=['Not assigned'] 
nbrNA['NBRVAL'] = D

#Dataframe contains 'Not assigned' value used for Comparison.
#print(nbrNA)

nbr1=nbr
#nbr1


#Compare with Each row if Neighborhood is Not Assigned add value from Borough.
for row_index,row in nbr.iterrows():
    if((nbr.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned') or (nbr.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned\n')):
        #print('inside if')
        nbr1.loc[row_index,['Neighborhood']] = nbr1.loc[row_index,['Borough']].values.astype('str') 
        
#Reset Index
#nbr.index = pd.RangeIndex(len(nbr.index))

#Uncomment below to view Dataframe nbr1 which contains Neighborhoods which were initially 'Not Assigned' now containing Borough values
print('nbr1')
nbr1

nbr1


,Postal_Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


##### More than one neighborhood can exist in one postal code area.

In [11]:
nbr2=nbr1.groupby('Postal_Code').agg({'Borough':'first',
                               'Neighborhood': ', '.join}).reset_index()
#Uncomment Below to view nbr2 which contains grouped Boroughs
nbr2

,Postal_Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae\n
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park\n, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West\n"
9,M1N,Scarborough,"Birch Cliff, Cliffside West\n"


##### Printing the number of rows of neighborhoods dataframe.

In [12]:
print(nbr2.shape)
print('nbr2')
nbr2

(103, 3)
nbr2


,Postal_Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae\n
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park\n, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West\n"
9,M1N,Scarborough,"Birch Cliff, Cliffside West\n"


### Filtering Neighborhood

Creating a copy of Dataframe. Dropping neighborhoods which do not Belong to Toronto

In [13]:
# define the dataframe columns
column_names = ['Postal_Code','Borough', 'Neighborhood'] 
# instantiate the dataframe
neighbor = pd.DataFrame(columns=column_names)

neighbor = nbr2.drop(nbr2[nbr2['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)

#Reset Index
neighbor.index = pd.RangeIndex(len(neighbor.index))
print('neighbor')
print(neighbor)

neighbor
   Postal_Code           Borough                                       Neighborhood
0          M4E      East Toronto                                        The Beaches
1          M4K      East Toronto                     The Danforth West\n, Riverdale
2          M4L      East Toronto                   The Beaches West\n, India Bazaar
3          M4M      East Toronto                                  Studio District\n
4          M4N   Central Toronto                                      Lawrence Park
5          M4P   Central Toronto                                 Davisville North\n
6          M4R   Central Toronto                               North Toronto West\n
7          M4S   Central Toronto                                       Davisville\n
8          M4T   Central Toronto                      Moore Park, Summerhill East\n
9          M4V   Central Toronto  Deer Park, Forest Hill SE\n, Rathnelly, South ...
10         M4W  Downtown Toronto                                   

In [14]:
#Keeping nbr_ungrp for ease of location calculations, using nbr1 by dropping Non Toronto Boroughs.
# define the dataframe columns
column_names = ['Postal_Code','Borough', 'Neighborhood'] 
# instantiate the dataframe
nbr_ungrp = pd.DataFrame(columns=column_names)

nbr_ungrp = nbr1.drop(nbr1[nbr1['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)

#Reset Index
nbr_ungrp.index = pd.RangeIndex(len(nbr_ungrp.index))

#Uncomment below to view Dataframe
print(nbr_ungrp)

   Postal_Code           Borough                                       Neighborhood
0          M5A  Downtown Toronto                                       Harbourfront
1          M5A  Downtown Toronto                                        Regent Park
2          M5B  Downtown Toronto                                           Ryerson

3          M5B  Downtown Toronto                                   Garden District

4          M5C  Downtown Toronto                                     St. James Town
5          M4E      East Toronto                                        The Beaches
6          M5E  Downtown Toronto                                        Berczy Park
7          M5G  Downtown Toronto                                Central Bay Street

8          M6G  Downtown Toronto                                          Christie

9          M5H  Downtown Toronto                                          Adelaide

10         M5H  Downtown Toronto                                            

#### Capturing Longitude And Latitude info of Locations

Using the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html

In [15]:
# uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge geopy --yes 
# convert an address into latitude and longitude values
import time
from geopy.geocoders import Nominatim

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

##### Looping through locations to fetch Latitude and Longitude

In [16]:
geolocator = Nominatim(user_agent="my_app app123")
#print('here')

for row_index, item in nbr_ungrp.iterrows():
    
    list1 = nbr_ungrp.loc[[row_index],['Neighborhood']].values.astype('str')
    loc = ' , Toronto, Ontario, Canada'
    list1.astype('str')
    list1 = np.append(list1, loc)
    latitude = None
    longitude = None
    location = None
    
    location = geolocator.geocode(list1 , limit = 15)
    #print(loc)
    #print(row_index)
    time.sleep(5)
    if(location is not None):
        #print(location)
        #print("location Address : ")
        #print(location.address)
        #print('latitude, longitude : ')
        #print((location.latitude, location.longitude))
        nbr_ungrp.loc[nbr_ungrp.index[row_index], 'Latitude'] = location.latitude
        nbr_ungrp.loc[nbr_ungrp.index[row_index], 'Longitude'] = location.longitude
        #print(nbr_ungrp[row_index])    

print(nbr_ungrp)

   Postal_Code           Borough                                       Neighborhood   Latitude  Longitude
0          M5A  Downtown Toronto                                       Harbourfront  43.640080 -79.380150
1          M5A  Downtown Toronto                                        Regent Park  43.660706 -79.360457
2          M5B  Downtown Toronto                                           Ryerson
  43.657474 -79.377905
3          M5B  Downtown Toronto                                   Garden District
  43.656500 -79.377114
4          M5C  Downtown Toronto                                     St. James Town  43.669403 -79.372704
5          M4E      East Toronto                                        The Beaches  43.671024 -79.296712
6          M5E  Downtown Toronto                                        Berczy Park  43.647989 -79.375393
7          M5G  Downtown Toronto                                Central Bay Street
        NaN        NaN
8          M6G  Downtown Toronto              

## Neighborhood analysis

### Importing Libraries for Foursqaue to analyse Neighborhood

In [17]:
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Importing to use the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Foursquare and map plotting Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


branca-0.3.1         | 25 KB     | #################################

In [18]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(nbr_ungrp['Borough'].unique()),
        nbr_ungrp.shape[0]
    )
)

#Dropping NANs
nbr_ungrp.dropna(inplace =True)
#Reset Index
nbr_ungrp.index = pd.RangeIndex(len(nbr_ungrp.index))
nbr_ungrp

The dataframe has 4 boroughs and 74 neighborhoods.


,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
1,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
2,M5B,Downtown Toronto,Ryerson,43.657474,-79.377905
3,M5B,Downtown Toronto,Garden District,43.656500,-79.377114
4,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704
5,M4E,East Toronto,The Beaches,43.671024,-79.296712
6,M5E,Downtown Toronto,Berczy Park,43.647989,-79.375393
7,M6G,Downtown Toronto,Christie,43.664111,-79.418405
8,M5H,Downtown Toronto,Adelaide,43.651923,-79.370879
9,M5H,Downtown Toronto,King,43.648949,-79.377754


Using geopy library to get the latitude and longitude values of Toronto City

In [19]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="my-app123")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a map of Toronto with neighborhoods superimposed on top

In [20]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(nbr_ungrp['Latitude'], nbr_ungrp['Longitude'], nbr_ungrp['Borough'], nbr_ungrp['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them

Define Foursquare Credentials and Version

In [41]:
CLIENT_ID = 'M5YWFTCLNYBN54LXSWEYYETLT2WWJM4OO1BNH2GD2NHFYBVM'
CLIENT_SECRET = '0DWUELCVOLNKHNFDYKFOYA4JTLXIGDSCXOTDNA40ZVHPEBML'
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: M5YWFTCLNYBN54LXSWEYYETLT2WWJM4OO1BNH2GD2NHFYBVM
CLIENT_SECRET:0DWUELCVOLNKHNFDYKFOYA4JTLXIGDSCXOTDNA40ZVHPEBML


### Let's explore the first neighborhood in our dataframe

Get the neighborhood's details

In [42]:
nbr_ungrp.loc[0]
neighborhood_latitude = np.float(nbr_ungrp.loc[0,['Latitude']].values)
neighborhood_longitude =  np.float(nbr_ungrp.loc[0,['Longitude']].values)

#### Now, let's get the top 25 venues that are in Harbourfront within a radius of 500 meters.

First, let's create the GET request URL. Naming the URL url.

In [46]:
radius = 500
results = requests.get(url).json()
results
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=M5YWFTCLNYBN54LXSWEYYETLT2WWJM4OO1BNH2GD2NHFYBVM&client_secret=0DWUELCVOLNKHNFDYKFOYA4JTLXIGDSCXOTDNA40ZVHPEBML&v=20180604&ll=43.653963,-79.387207&radius=500&limit=100'

get_category_type function

In [47]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Cleaning the json and structuring it into a pandas dataframe.

In [48]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Japango,Sushi Restaurant,43.655268,-79.385165
2,Textile Museum of Canada,Art Museum,43.654396,-79.386500
3,Poke Guys,Poke Place,43.654895,-79.385052
4,Sansotei Ramen 三草亭,Ramen Restaurant,43.655157,-79.386501


How many venues were returned by Foursquare?

In [49]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

72 venues were returned by Foursquare.


### Exploring all Neighborhoods

In [50]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [51]:
toronto_venues = getNearbyVenues(names=nbr_ungrp['Neighborhood'],
                                   latitudes=nbr_ungrp['Latitude'],
                                   longitudes=nbr_ungrp['Longitude']
                                  )

Harbourfront
Regent Park
Ryerson

Garden District

St. James Town
The Beaches
Berczy Park
Christie

Adelaide

King

Richmond

Dovercourt Village
Dufferin

Harbourfront East

Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West

Riverdale
Design Exchange
Toronto Dominion Centre
Brockton

Exhibition Place
Parkdale Village
The Beaches West

India Bazaar
Commerce Court
Lawrence Park
Roselawn

Davisville North

Forest Hill North
High Park
The Annex
Yorkville
Parkdale
Roncesvalles
Davisville

Harbord

University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East

Chinatown
Grange Park
Kensington Market
Deer Park
Rathnelly
South Hill
Summerhill West

CN Tower
Bathurst Quay

Harbourfront West

King and Spadina
South Niagara
Rosedale
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and Wellesley


##### Size of dataframe

In [52]:
print(toronto_venues.shape)
toronto_venues.head()

(3453, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.64008,-79.38015,Harbour Square Park,43.639253,-79.378395,Park
1,Harbourfront,43.64008,-79.38015,Lake Ontario,43.638945,-79.379665,Lake
2,Harbourfront,43.64008,-79.38015,Harbourfront,43.639526,-79.380688,Neighborhood
3,Harbourfront,43.64008,-79.38015,BeaverTails,43.639736,-79.380068,Bakery
4,Harbourfront,43.64008,-79.38015,Miku,43.641374,-79.377531,Japanese Restaurant


Let's check how many venues were returned for each neighborhood

In [53]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,93,93,93,93,93,93
Bathurst Quay,25,25,25,25,25,25
Berczy Park,100,100,100,100,100,100
Brockton,19,19,19,19,19,19
CN Tower,93,93,93,93,93,93
Cabbagetown,50,50,50,50,50,50
Chinatown,100,100,100,100,100,100
Christie,59,59,59,59,59,59
Church and Wellesley,87,87,87,87,87,87


Let's find out how many unique categories can be curated from all the returned venues

In [54]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 282 uniques categories.


### 3. Analyze Each Neighborhood

In [55]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Accessories Store,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tunnel,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Win

In [56]:
toronto_onehot.shape

(3453, 282)

Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [57]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,Accessories Store,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tunnel,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Win

In [58]:
toronto_grouped.shape

(61, 282)

Let's print each neighborhood along with the top 5 most common venues

In [59]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide
----
                venue  freq
0         Coffee Shop  0.10
1  Italian Restaurant  0.05
2                Café  0.04
3          Restaurant  0.04
4       Grocery Store  0.03


----Bathurst Quay
----
                  venue  freq
0           Coffee Shop  0.20
1                  Café  0.12
2                  Park  0.08
3                   Pub  0.04
4  Caribbean Restaurant  0.04


----Berczy Park----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.06
2   Restaurant  0.05
3        Hotel  0.05
4       Bakery  0.04


----Brockton
----
                       venue  freq
0                        Bar  0.16
1                       Park  0.11
2      Vietnamese Restaurant  0.11
3          Korean Restaurant  0.05
4  South American Restaurant  0.05


----CN Tower----
                venue  freq
0               Hotel  0.09
1         Coffee Shop  0.09
2  Italian Restaurant  0.05
3         Pizza Place  0.05
4            Aquarium  0.04


----Cabbagetown----
                 venue

##### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 7 venues for each neighborhood.

In [61]:
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Adelaide,Coffee Shop,Italian Restaurant,Restaurant,Café,Grocery Store,Pizza Place,Bakery
1,Bathurst Quay,Coffee Shop,Café,Park,Light Rail Station,Ramen Restaurant,Pub,Japanese Restaurant
2,Berczy Park,Coffee Shop,Café,Restaurant,Hotel,Bakery,Gastropub,Gym
3,Brockton,Bar,Vietnamese Restaurant,Park,Bakery,Grocery Store,French Restaurant,Light Rail Station
4,CN Tower,Coffee Shop,Hotel,Pizza Place,Italian Restaurant,Scenic Lookout,Gym,Aquarium
5,Cabbagetown,Restaurant,Café,Coffee Shop,Indian Restaurant,Gastropub,Bakery,Japanese Restaurant
6,Chinatown,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Vietnamese Restaurant,Bar,Dumpling Restaurant,Mexican Restaurant
7,Christie,Korean Restaurant,Indian Restaurant,Coffee Shop,Grocery Store,Café,Sandwich Place,Karaoke Bar
8,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Burger Joint,Hotel
9,Commerce Court,Coffee Shop,Hotel,Café,American Restaurant,Restaurant,Bakery,Gastropub


### 4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 4 clusters

In [62]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',1)
#print(toronto_grouped_clustering)
#print(toronto_grouped)
# run k-means clustering
kmeans = KMeans(init = "k-means++", n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
labels = kmeans.labels_[0:63] 
print(labels)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
 1 1 1 3 1 1 1 1 1 1 1 1 3 1 1 1 1 1 2 1 1 1 1 1]


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [63]:
toronto_merged = nbr_ungrp
print(toronto_merged.shape)
labels = np.append(labels,labels[0])
print(labels.shape)
# add clustering labels
toronto_merged['Cluster Labels'] = labels.tolist()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

(62, 5)
(62,)


,Postal_Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150,1,Coffee Shop,Café,Restaurant,Hotel,Pizza Place,Italian Restaurant,Bakery
1,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,1,Coffee Shop,Thai Restaurant,Pharmacy,Grocery Store,Food Truck,Beer Store,Electronics Store
2,M5B,Downtown Toronto,Ryerson,43.657474,-79.377905,1,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Theater,Tea Room
3,M5B,Downtown Toronto,Garden District,43.656500,-79.377114,1,Clothing Store,Coffee Shop,Cosmetics Shop,Restaurant,Hotel,Fast Food Restaurant,Ramen Restaurant
4,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704,1,Coffee Shop,Pizza Place,Café,Grocery Store,Dive Bar,Caribbean Restaurant,Beer Store


Visualize the resulting clusters

In [64]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 5. Examine Clusters

###### Cluster 1

In [65]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
18,East Toronto,0,Coffee Shop,Bus Line,Metro Station,Skating Rink,Sushi Restaurant,Fried Chicken Joint,Mexican Restaurant
32,West Toronto,0,Baseball Field,Dog Run,Skating Rink,Scenic Lookout,Other Great Outdoors,Garden,Café


##### Cluster 2

In [66]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Pizza Place,Italian Restaurant,Bakery
1,Downtown Toronto,1,Coffee Shop,Thai Restaurant,Pharmacy,Grocery Store,Food Truck,Beer Store,Electronics Store
2,Downtown Toronto,1,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Theater,Tea Room
3,Downtown Toronto,1,Clothing Store,Coffee Shop,Cosmetics Shop,Restaurant,Hotel,Fast Food Restaurant,Ramen Restaurant
4,Downtown Toronto,1,Coffee Shop,Pizza Place,Café,Grocery Store,Dive Bar,Caribbean Restaurant,Beer Store
5,East Toronto,1,Beach,Pizza Place,Bakery,Park,Coffee Shop,Pub,Restaurant
6,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Bakery,Gastropub,Gym
7,Downtown Toronto,1,Korean Restaurant,Indian Restaurant,Coffee Shop,Grocery Store,Café,Sandwich Place,Karaoke Bar
8,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Restaurant,Café,Grocery Store,Pizza Place,Bakery
9,Downtown Toronto,1,Coffee Shop,Restaurant,Hotel,Italian Restaurant,Café,Gastropub,American Restaurant


###### Cluster 3

In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
55,Downtown Toronto,2,Bakery,Restaurant,Coffee Shop,Park,Spa,Sandwich Place,Asian Restaurant


###### Cluster 4

In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
40,West Toronto,3,Pizza Place,Rental Car Location,Coffee Shop,Bank,Burger Joint,Thai Restaurant,Athletics & Sports
49,Central Toronto,3,Sandwich Place,Coffee Shop,Café,Pizza Place,History Museum,Historic Site,Pub
